In [ ]:

from collections import defaultdict, Counter
import numpy as np
import pandas as pd

class LanguageNgramModel:
  
    def __init__(self, order=1, smoothing=1.0, recursive=0.001):
        self.order = order
        self.smoothing = smoothing
        self.recursive = recursive
    
    def fit(self, corpus):
      
        self.counter_ = defaultdict(lambda: Counter())
        self.vocabulary_ = set()
        for i, token in enumerate(corpus[self.order:]):
            context = corpus[i:(i+self.order)]
            self.counter_[context][token] += 1
            self.vocabulary_.add(token)
        self.vocabulary_ = sorted(list(self.vocabulary_))
        if self.recursive > 0 and self.order > 0:
            self.child_ = LanguageNgramModel(self.order-1, self.smoothing, self.recursive)
            self.child_.fit(corpus)
            
    def get_counts(self, context):
    
        if self.order:
            local = context[-self.order:]
        else:
            local = ''
        freq_dict = self.counter_[local]
        freq = pd.Series(index=self.vocabulary_)
        for i, token in enumerate(self.vocabulary_):
            freq[token] = freq_dict[token] + self.smoothing
        if self.recursive > 0 and self.order > 0:
            child_freq = self.child_.get_counts(context) * self.recursive
            freq += child_freq
        return freq
    
    def predict_proba(self, context):

        counts = self.get_counts(context)
        return counts / counts.sum()
    
    def single_log_proba(self, context, continuation):
 
        result = 0.0
        for token in continuation:
            result += np.log(self.predict_proba(context)[token])
            context += token
        return result
    
    def single_proba(self, context, continuation):

        return np.exp(self.single_log_proba(context, continuation))



In [ ]:
class MissingLetterModel:
  
    def __init__(self, order=0, smoothing_missed=0.3, smoothing_total=1.0):
        self.order = order
        self.smoothing_missed = smoothing_missed
        self.smoothing_total = smoothing_total
    
    def fit(self, sentence_pairs):
     
        self.missed_counter_ = defaultdict(lambda: Counter())
        self.total_counter_ = defaultdict(lambda: Counter())
        for (original, observed) in sentence_pairs:
            for i, (original_letter, observed_letter) \
                    in enumerate(zip(original[self.order:], observed[self.order:])):
                context = original[i:(i+self.order)]
                if observed_letter == '-':
                    self.missed_counter_[context][original_letter] += 1
                self.total_counter_[context][original_letter] += 1 
    
    def predict_proba(self, context, last_letter):

        if self.order:
            local = context[-self.order:]
        else:
            local = ''
        missed_freq = self.missed_counter_[local][last_letter] + self.smoothing_missed
        total_freq = self.total_counter_[local][last_letter] + self.smoothing_total
        return missed_freq / total_freq
    
    def single_log_proba(self, context, continuation, actual=None):
    
        if not actual:
            actual = continuation
        result = 0.0
        for orig_token, act_token in zip(continuation, actual):
            pp = self.predict_proba(context, orig_token)
            if act_token != '-':
                pp = 1 - pp
            result += np.log(pp)
            context += orig_token
        return result
    
    def single_proba(self, context, continuation, actual=None):
     
        return np.exp(self.single_log_proba(context, continuation, actual))

lang_model = LanguageNgramModel(1)
lang_model.fit(' abracadabra ')
print(lang_model.predict_proba(' bra'))

missed_model = MissingLetterModel(0)
missed_model.fit([('abracadabra', 'abr-c-d-br-')]) 

print({letter: missed_model.predict_proba('abr', letter) for letter in 'abc'})


print(missed_model.single_proba('', 'abra', 'abr-'))

np.log10(27) * 10


     0.181777
a    0.091297
b    0.272529
c    0.181686
d    0.181686
r    0.091025
dtype: float64
{'a': 0.7166666666666667, 'b': 0.09999999999999999, 'c': 0.15}
0.16447499999999998


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:33: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.


14.313637641589875

In [ ]:

from heapq import heappush, heappop

def generate_options(prefix_proba, prefix, suffix, 
                     lang_model, missed_model, optimism=0.5, cache=None):
   
    options = []
    for letter in lang_model.vocabulary_ + ['']:
        if letter:  # here we assume the character was missing
            next_letter = letter
            new_suffix = suffix
            new_prefix = prefix + next_letter
            proba_missing_state = - np.log(missed_model.predict_proba(prefix, letter))
        else:  # here we assume there was no missing character
            next_letter = suffix[0]
            new_suffix = suffix[1:]
            new_prefix = prefix + next_letter
            proba_missing_state = - np.log((1 - missed_model.predict_proba(prefix, next_letter)))
        proba_next_letter = - np.log(lang_model.single_proba(prefix, next_letter))
        if cache:
            proba_suffix = cache[len(new_suffix)] * optimism
        else:
            proba_suffix = - np.log(lang_model.single_proba(new_prefix, new_suffix)) * optimism
        proba = prefix_proba + proba_next_letter + proba_missing_state + proba_suffix
        options.append((proba, new_prefix, new_suffix, letter, proba_suffix))
    return options

print(generate_options(0, ' ', 'brac ', lang_model, missed_model))


def noisy_channel(word, lang_model, missed_model, freedom=3.0, 
                  max_attempts=10000, optimism=0.9, verbose=False):

    query = word + ' '
    prefix = ' '
    prefix_proba = 0.0
    suffix = query
    full_origin_logprob = -lang_model.single_log_proba(prefix, query)
    no_missing_logprob = -missed_model.single_log_proba(prefix, query)
    best_logprob = full_origin_logprob + no_missing_logprob
    # add an empty prefix to the heap
    heap = [(best_logprob * optimism, prefix, suffix, '', best_logprob * optimism)]
    # add the default candidate (without missing characters) 
    candidates = [(best_logprob, prefix + query, '', None, 0.0)]
    if verbose:
        print('baseline score is', best_logprob)
    # prepare storage of the phrase suffix probabilities
    cache = {}
    for i in range(len(query)+1):
        future_suffix = query[:i]
        cache[len(future_suffix)] = -lang_model.single_log_proba('', future_suffix) # rough approximation
        cache[len(future_suffix)] += -missed_model.single_log_proba('', future_suffix) # at least add missingness
    
    for i in range(max_attempts):
        if not heap:
            break
        next_best = heappop(heap)
        if verbose:
            print(next_best)
        if next_best[2] == '':  # the phrase is fully decoded
            # if the phrase is good enough, add it to the answer
            if next_best[0] <= best_logprob + freedom:
                candidates.append(next_best)
                # update estimate of the best likelihood
                if next_best[0] < best_logprob:
                    best_logprob = next_best[0]
        else: # # the phrase is not fully decoded - generate more options
            prefix_proba = next_best[0] - next_best[4] # all proba estimate minus suffix
            prefix = next_best[1]
            suffix = next_best[2]
            new_options = generate_options(
                prefix_proba, prefix, suffix, lang_model, 
                missed_model, optimism, cache)
            # add only the solution potentioally no worse than the best + freedom
            for new_option in new_options: 
                if new_option[0] < best_logprob + freedom:
                    heappush(heap, new_option)
    if verbose:
        print('heap size is', len(heap), 'after', i, 'iterations')
    result = {}
    for candidate in candidates:
        if candidate[0] <= best_logprob + freedom:
            result[candidate[1][1:-1]] = candidate[0]
    return result


result = noisy_channel('brc', lang_model, missed_model, verbose=True, freedom=1)
print(result)




[(6.929663174828117, '  ', 'brac ', ' ', 3.7800651217336947), (5.042879645338754, ' a', 'brac ', 'a', 3.4572571306016755), (8.09487194753453, ' b', 'brac ', 'b', 3.846661605771999), (7.623807861705187, ' c', 'brac ', 'c', 3.7800651217336947), (7.623807861705187, ' d', 'brac ', 'd', 3.7800651217336947), (8.09487194753453, ' r', 'brac ', 'r', 3.846661605771999), (4.858238261775765, ' b', 'rac ', '', 2.8072524973494524)]
baseline score is 7.683183062275049
(6.914864756047544, ' ', 'brc ', '', 6.914864756047544)
(6.755450684372974, ' b', 'rc ', '', 4.704464919946662)
(5.824911949460505, ' br', 'c ', '', 2.686363732552668)
(7.088440394887126, ' brc', ' ', '', 1.7075575253192956)
(7.139259830483152, ' bra', 'c ', 'a', 2.686363732552668)
(7.68318306227505, ' brc ', '', '', -0.0)
(8.028446927360166, ' brac', ' ', '', 1.7075575253192956)
(8.362157608120238, ' a', 'brc ', 'a', 6.776535093383159)
(7.695457216846014, ' ab', 'rc ', '', 4.704464919946662)
(6.764918481933545, ' abr', 'c ', '', 2.6863

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:33: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.


In [ ]:
from google.colab import files

uploaded = files.upload()

Saving 01 - The Fellowship Of The Ring.txt to 01 - The Fellowship Of The Ring.txt


In [ ]:
import re
# read the text
with open('/content/infosys-ar-21.txt', encoding = "ISO-8859-1") as f:
    text = f.read()
# leave only letters and spaces in the text
text2 = re.sub(r'[^a-z ]+', '', text.lower().replace('\n', ' '))
all_letters = ''.join(list(sorted(list(set(text2)))))
print(repr(all_letters)) # ' abcdefghijklmnopqrstuvwxyz'
# Prepare training sample for the abbreviation model 
missing_set =  (
    [(all_letters, '-' * len(all_letters))] * 3 # all chars missing
    + [(all_letters, all_letters)] * 10 # all chars are NOT missing
    + [('aeiouy', '------')] * 30 # only vowels are missing
)
# Train the both models
big_lang_m = LanguageNgramModel(order=4, smoothing=0.001, recursive=0.01)
big_lang_m.fit(text2)
big_err_m = MissingLetterModel(order=0, smoothing_missed=0.1)
big_err_m.fit(missing_set)


for i in range(5):
    tmp = LanguageNgramModel(i, 0.001, 0.01)
    tmp.fit(text2[0:-5000])
    print(i, tmp.single_log_proba(' ', text2[-5000:]))



In [ ]:
noisy_channel('ifsy', big_lang_m, big_err_m)

noisy_channel('rvn', big_lang_m, big_err_m)

noisy_channel('opngblnc', big_lang_m, big_err_m)

noisy_channel('drvt', big_lang_m, big_err_m)

noisy_channel('fnclassts', big_lang_m, big_err_m)

noisy_channel('dprctn', big_lang_m, big_err_m)

print(noisy_channel('blnc', big_lang_m, big_err_m))

print(noisy_channel('dvdnd', big_lang_m, big_err_m, freedom=5))

print(noisy_channel('lblts', big_lang_m, big_err_m, freedom=5))

part = text[10502:11149]
result = ''
for i, letter in enumerate(part):
    if np.random.rand() * 0.5 < big_err_m.single_proba(part[0:i], letter):
        result += letter
print(result)



/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:33: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.


{'balance': 9.554093699594214}
{'dividend': 11.475488678567428, 'dividend ': 14.738236263601777, 'dividends': 13.894727405636457}
{'liabilities': 10.272056457979586, 'liabilities ': 13.596525807258878}

This year to appointment as at march   flucture company is requity inc designificational asset the preferred to joining busing any below hedge volune with and key member  exercise during its at of the difference received to to the trusts our yrealize our emainik satisfyiness methods and financial as and exte.
